<a id='0.1'></a>
## 0.1 load required packages

In [1]:
%run "..\..\..\Startup_py3.py"
sys.path.append(r"..\..\..\..\..\Documents")

import ImageAnalysis3 as ia
%matplotlib notebook

from ImageAnalysis3 import *
print(os.getpid())

# library design specific tools
from ImageAnalysis3.library_tools import LibraryDesigner as ld
from ImageAnalysis3.library_tools import LibraryTools as lt
# biopython imports
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Blast.Applications import NcbiblastnCommandline
from Bio.Blast import NCBIXML

27540


In [2]:
import pandas as pd

<a id='0.2'></a>
## 0.2 folders

In [125]:
## Some folders
# human genome
reference_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\Genomes\mouse\GRCm38_ensembl'
genome_folder = os.path.join(reference_folder, 'Genome')
# Library directories
pool_folder = r'\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain'
# folder for sub-pool
library_folder = os.path.join(pool_folder, f'mouse_genome_1000')

In [126]:
# laod encoding
# summarize total readout usage
encoding_folder = os.path.join(pool_folder, f'Encoding_design')
region_2_bits = pickle.load(open(os.path.join(encoding_folder, 'region_2_bits.pkl'), 'rb'))

In [127]:
# load probes
probes = []
final_pb_filename = os.path.join(pool_folder, 'final_pool_probes.fasta')

with open(final_pb_filename, 'r') as handle:
    for record in SeqIO.parse(handle, "fasta"):
        probes.append(record)

In [25]:
for _pb in probes:
    _reg = _pb.id.split('loc_')[1].split('_')[0]
    _start, _end = _reg.split(':')[1].split('-')
    # gene info
    _gene = _pb.id.split('gene_')[1].split('_')[0]
    _pb_pos = int(_pb.id.split('pos_')[1].split('_')[0]) + int(_start)
    
    break

In [26]:
_pb_pos

3742742

In [23]:
_start

'3740000'

In [19]:
int(_pb_pos)

2742

In [28]:
# load region names and real coordinates
region_2_names = {}
region_2_coords = {}
for _pb in probes:
    _reg = _pb.id.split('loc_')[1].split('_')[0]
    _start, _end = _reg.split(':')[1].split('-')
    # gene info
    _gene = _pb.id.split('gene_')[1].split('_')[0]
    _pb_pos = int(_pb.id.split('pos_')[1].split('_')[0]) + int(_start)
    
    if _gene not in region_2_names:
        region_2_names[_gene] = _reg
    if _gene not in region_2_coords:
        region_2_coords[_gene] = [_pb_pos]
    else:
        region_2_coords[_gene].append(_pb_pos)
print(region_2_names)

{'1': '1:3740000-3760000', '3': '1:8740000-8760000', '5': '1:13740000-13760000', '7': '1:18740000-18760000', '9': '1:23740000-23760000', '11': '1:28740000-28760000', '13': '1:33740000-33760000', '15': '1:38740000-38760000', '17': '1:43740000-43760000', '19': '1:48740000-48760000', '21': '1:53740000-53760000', '23': '1:58740000-58760000', '25': '1:63740000-63760000', '27': '1:68740000-68760000', '29': '1:73740000-73760000', '31': '1:78740000-78760000', '33': '1:83740000-83760000', '35': '1:88740000-88760000', '37': '1:93740000-93760000', '39': '1:98740000-98760000', '41': '1:103740000-103760000', '43': '1:108740000-108760000', '45': '1:113740000-113760000', '47': '1:118740000-118760000', '49': '1:123740000-123760000', '51': '1:128740000-128760000', '53': '1:133740000-133760000', '55': '1:138740000-138760000', '57': '1:143740000-143760000', '59': '1:148740000-148760000', '61': '1:153740000-153760000', '63': '1:158740000-158760000', '65': '1:163740000-163760000', '67': '1:168740000-168760

In [43]:
# generate dict
pb_len = 42
min_probe_num = 50

region_2_probeInfo = {}
for _gene in region_2_coords:
    _name = region_2_names[_gene]
    _chr = _name.split(':')[0]
    _coords = region_2_coords[_gene]
    if len(_coords) < min_probe_num:
        continue
    _info = f"{_chr}:{np.min(_coords)}-{np.max(_coords)+pb_len}"
    region_2_probeInfo[_gene] = _info

In [8]:
# laod encoding
# summarize total readout usage
region_2_readout_dict = pickle.load(open(os.path.join(library_folder, 'region_2_readout.pkl'), 'rb'))
print(region_2_readout_dict)

{1: ['c36', 'c62', 'c77'], 2: ['c17', 'c42', 'c86'], 3: ['c34', 'c72', 'c82'], 4: ['c57', 'c58', 'c94'], 5: ['c41', 'c78', 'c93'], 6: ['c61', 'c65', 'c80'], 7: ['c66', 'c74', 'c83'], 8: ['c23', 'c44', 'c54'], 9: ['c4', 'c45', 'c90'], 10: ['c51', 'c64', 'c75'], 11: ['c12', 'c67', 'c69'], 12: ['c15', 'c43', 'c68'], 13: ['c30', 'c31', 'c62'], 14: ['c10', 'c22', 'c47'], 15: ['c5', 'c6', 'c13'], 16: ['c3', 'c35', 'c49'], 17: ['c39', 'c84', 'c88'], 18: ['c8', 'c20', 'c56'], 19: ['c24', 'c32', 'c55'], 20: ['c18', 'c29', 'c73'], 21: ['c53', 'c89', 'c91'], 22: ['c7', 'c14', 'c63'], 23: ['c0', 'c37', 'c81'], 24: ['c70', 'c95', 'c97'], 25: ['c26', 'c28', 'c60'], 26: ['c11', 'c68', 'c74'], 27: ['c21', 'c56', 'c85'], 28: ['c54', 'c83', 'c87'], 29: ['c2', 'c76', 'c79'], 30: ['c48', 'c51', 'c96'], 31: ['c8', 'c44', 'c65'], 32: ['c1', 'c19', 'c53'], 33: ['c5', 'c27', 'c33'], 34: ['c67', 'c92', 'c98'], 35: ['c10', 'c52', 'c58'], 36: ['c12', 'c34', 'c42'], 37: ['c4', 'c38', 'c77'], 38: ['c40', 'c59', 'c

In [36]:
# load used readouts
readout_usage_file = os.path.join(library_folder, 'readout_usage.pkl')
readout_dict = pickle.load(open(readout_usage_file, 'rb'))

In [37]:
readout_dict

{'c': [SeqRecord(seq=Seq('AACCACGGGCTAGAAATATCCGAGTGGCAT'), id='NDB_784', name='NDB_784', description='NDB_784', dbxrefs=[]),
  SeqRecord(seq=Seq('GCCTAAGACGTGCGACATGTTATGCTAGGC'), id='NDB_755', name='NDB_755', description='NDB_755', dbxrefs=[]),
  SeqRecord(seq=Seq('GCCGTTTCGTATATGTCGATGTCCTTAGAC'), id='NDB_826', name='NDB_826', description='NDB_826', dbxrefs=[]),
  SeqRecord(seq=Seq('TACAAATACCGGTGGTTGTCACTTTCCGAA'), id='NDB_713', name='NDB_713', description='NDB_713', dbxrefs=[]),
  SeqRecord(seq=Seq('TCCGTGACGAGATCCCGGTCGGATATGTGA'), id='NDB_865', name='NDB_865', description='NDB_865', dbxrefs=[]),
  SeqRecord(seq=Seq('GAGCTTATGACTACCACGGCATCGATTTAG'), id='NDB_725', name='NDB_725', description='NDB_725', dbxrefs=[]),
  SeqRecord(seq=Seq('CGCGAAGATCGAACTCGTTGACTAAGCATT'), id='NDB_817', name='NDB_817', description='NDB_817', dbxrefs=[]),
  SeqRecord(seq=Seq('GTGGTAAACCTCAACTACTAAGCCGCGGTA'), id='NDB_710', name='NDB_710', description='NDB_710', dbxrefs=[]),
  SeqRecord(seq=Seq('CCAGTA

## bit_2_readout

In [38]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
dtype_dict = {_v:_k for _k,_v in ia.classes._allowed_kwds.items()}

In [26]:
for _dtype, _records in readout_dict.items():
    
    bit_readout_df = pd.DataFrame([[_i+1, _r.id] for _i, _r in enumerate(_records)], columns=['Bit','ReadoutName'])
    save_filename = os.path.join(save_folder, f'CTP11-mouse-genome-1000_{dtype_dict[_dtype]}_readouts.csv')
    print(save_filename)
    bit_readout_df.to_csv(save_filename, index=False)

\\10.245.74.212\Chromatin_NAS_2\Chromatin_Libraries\CTP-11_brain\Summary_tables\CTP11-mouse-genome-1000_combo_readouts.csv


In [25]:
bit_readout_df

,Bit,ReadoutName
0,1,NDB_784
1,2,NDB_755
2,3,NDB_826
3,4,NDB_713
4,5,NDB_865
...,...,...
94,95,NDB_940
95,96,NDB_872
96,97,NDB_934
97,98,NDB_926


# Codebook

In [39]:
region_2_readout_dict

{1: ['c36', 'c62', 'c77'],
 2: ['c17', 'c42', 'c86'],
 3: ['c34', 'c72', 'c82'],
 4: ['c57', 'c58', 'c94'],
 5: ['c41', 'c78', 'c93'],
 6: ['c61', 'c65', 'c80'],
 7: ['c66', 'c74', 'c83'],
 8: ['c23', 'c44', 'c54'],
 9: ['c4', 'c45', 'c90'],
 10: ['c51', 'c64', 'c75'],
 11: ['c12', 'c67', 'c69'],
 12: ['c15', 'c43', 'c68'],
 13: ['c30', 'c31', 'c62'],
 14: ['c10', 'c22', 'c47'],
 15: ['c5', 'c6', 'c13'],
 16: ['c3', 'c35', 'c49'],
 17: ['c39', 'c84', 'c88'],
 18: ['c8', 'c20', 'c56'],
 19: ['c24', 'c32', 'c55'],
 20: ['c18', 'c29', 'c73'],
 21: ['c53', 'c89', 'c91'],
 22: ['c7', 'c14', 'c63'],
 23: ['c0', 'c37', 'c81'],
 24: ['c70', 'c95', 'c97'],
 25: ['c26', 'c28', 'c60'],
 26: ['c11', 'c68', 'c74'],
 27: ['c21', 'c56', 'c85'],
 28: ['c54', 'c83', 'c87'],
 29: ['c2', 'c76', 'c79'],
 30: ['c48', 'c51', 'c96'],
 31: ['c8', 'c44', 'c65'],
 32: ['c1', 'c19', 'c53'],
 33: ['c5', 'c27', 'c33'],
 34: ['c67', 'c92', 'c98'],
 35: ['c10', 'c52', 'c58'],
 36: ['c12', 'c34', 'c42'],
 37: ['c4', 

In [76]:
# generate the codebook
codebook = pd.DataFrame(columns=['name','id']+[_r.id for _r in readout_dict['c']])
# decide region ids first
codebook['id'] = sorted(list(region_2_probeInfo.keys()), key=lambda v:int(v))
# region names
codebook['name'] = [region_2_probeInfo[str(_gene)] for _gene in codebook['id']]
# code
max_bit = np.max([np.max([int(_b[1:]) for _b in _bits]) for _gname, _bits in region_2_readout_dict.items()]) + 1
for _gene in codebook['id']:
    _bits = region_2_readout_dict[int(_gene)]
    binary_code = []
    for _i in range(max_bit):
        if f"c{_i}" in _bits:
            binary_code.append(1)
        else:
            binary_code.append(0)
    #print(_gene)
    codebook.loc[codebook['id']==_gene, codebook.columns[2:]] = binary_code
    #codebook.loc[codebook['name']==_gene,'id'] = int(_bits[0].split('u')[1])

In [77]:
codebook

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_1028,NDB_925,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910
0,1:3742742-3759944,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1:6245958-6258969,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1:8740008-8759916,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,1:11247744-11257616,4,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1:13741888-13757922,5,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
983,X:161247067-161259813,1057,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
984,X:163750534-163758706,1058,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
985,X:166247682-166259932,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
986,X:168746045-168757590,1060,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [78]:
# sanity check
codebook.values[:,2:].sum(1)

array([3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
       3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,

In [100]:
# add chromosome info
codebook['chr'] = [_n.split(':')[0] for _n in codebook['name']]
for _chr in np.unique(codebook['chr']):
    _chr_codebook = codebook.loc[codebook['chr']==_chr]
    _chr_reg_info = [np.mean([int(_loc) for _loc in _n.split(':')[1].split('-')]) for _n in _chr_codebook['name']]
    _chr_orders = np.array(_chr_reg_info).argsort().argsort().astype(np.int32)
    codebook.loc[codebook['chr']==_chr, 'chr_order'] = _chr_orders
# sort id
codebook.sort_values('id', inplace=True)
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-1000_codebook_probe.csv')

In [101]:
codebook.to_csv(save_filename,index=None)

In [107]:
pickle.load(open(r'E:\Users\puzheng\Downloads\endnote\cell_info.pkl', 'rb'))

{'analysis_folder': 'Y:\\20190302_IMR90_CTP05_intronFISH\\Analysis',
 'annotated_folders': ['Y:\\20190302_IMR90_CTP05_intronFISH\\H0R0',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H1R1',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H2R2',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H3R3',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H4R4',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H5R5',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H6R6',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H7R7',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H8R8',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H9R9',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H10R10',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H11R11',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H12R12',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H13R13',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H14R14',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H15R15',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H16R16',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H17R17',
  'Y:\\20190302_IMR90_CTP05_intronFISH\\H18R18',

In [108]:
for _i in range(13,25):
    print(f"/dev/sas{_i}p3", end=' ')

/dev/sas13p3 /dev/sas14p3 /dev/sas15p3 /dev/sas16p3 /dev/sas17p3 /dev/sas18p3 /dev/sas19p3 /dev/sas20p3 /dev/sas21p3 /dev/sas22p3 /dev/sas23p3 /dev/sas24p3 

# generate 500 gene codebook

In [113]:
codebook_500 = pd.DataFrame(columns=codebook.columns)

# extract 500 genes
sel_regions_500 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_500':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_500:
            sel_regions_500.append(_gene)
            
for _gene in sel_regions_500:
    codebook_500 = codebook_500.append(codebook.loc[codebook['id']==_gene])
#codebook_500.sort_values('id', inplace=True)
codebook_500

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
0,1:3742742-3759944,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,0.0
2,1:8740008-8759916,3,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,2.0
4,1:13741888-13757922,5,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,1,4.0
6,1:18749330-18759924,7,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,6.0
8,1:23749258-23759965,9,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,1,8.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
979,X:151244970-151259694,1053,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,48.0
981,X:156242675-156257265,1055,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,50.0
983,X:161247067-161259813,1057,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,52.0
985,X:166247682-166259932,1059,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,X,54.0


In [114]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-500_codebook.csv')

codebook_500.to_csv(save_filename, index=None)

# 250-1

In [115]:
codebook_250_1 = pd.DataFrame(columns=codebook.columns)

# extract 250_1 genes
sel_regions_250_1 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_250_1':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_250_1:
            sel_regions_250_1.append(_gene)
            
for _gene in sel_regions_250_1:
    codebook_250_1 = codebook_250_1.append(codebook.loc[codebook['id']==_gene])
#codebook_250_1.sort_values('id', inplace=True)
codebook_250_1

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
1,1:6245958-6258969,2,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1.0
5,1:16251322-16259969,6,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,5.0
9,1:26243246-26258573,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,9.0
13,1:36250053-36259834,14,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,13.0
17,1:46240257-46258868,18,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,17.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
968,X:116240181-116259555,1039,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,37.0
972,X:131245946-131259984,1045,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,41.0
976,X:141244904-141260001,1049,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,45.0
980,X:153743904-153759903,1054,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,49.0


In [116]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-250_1_codebook.csv')

codebook_250_1.to_csv(save_filename, index=None)

# 250-2

In [117]:
codebook_250_2 = pd.DataFrame(columns=codebook.columns)

# extract 250_2 genes
sel_regions_250_2 = []
for _pb in probes:
    if _pb.id.split('library_')[-1] == '1000-mouse-genome_250_2':
        _gene = _pb.id.split('gene_')[1].split('_')[0]
        if _gene not in sel_regions_250_2:
            sel_regions_250_2.append(_gene)
            
for _gene in sel_regions_250_2:
    codebook_250_2 = codebook_250_2.append(codebook.loc[codebook['id']==_gene])
#codebook_250_2.sort_values('id', inplace=True)
codebook_250_2

,name,id,NDB_784,NDB_755,NDB_826,NDB_713,NDB_865,NDB_725,NDB_817,NDB_710,...,NDB_824,NDB_712,NDB_818,NDB_940,NDB_872,NDB_934,NDB_926,NDB_910,chr,chr_order
3,1:11247744-11257616,4,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,1,3.0
7,1:21249750-21259997,8,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,7.0
11,1:31240000-31259970,12,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,11.0
15,1:41248518-41259654,16,0,0,0,1,0,0,0,0,...,0,0,0,0,0,0,0,0,1,15.0
19,1:51245791-51259984,20,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,19.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,X:121240154-121259878,1041,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,X,39.0
974,X:136250172-136259999,1047,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,43.0
978,X:146246596-146260000,1051,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,X,47.0
982,X:158749404-158759978,1056,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,X,51.0


In [118]:
save_folder = os.path.join(pool_folder, 'Summary_tables')
save_filename = os.path.join(save_folder, 'CTP11-mouse-genome-250_2_codebook.csv')

codebook_250_2.to_csv(save_filename, index=None)